In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import country_converter as coco
import xlrd
import pickle
import os
import math
import time
from pandas.tseries.offsets import MonthEnd
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS
from statsmodels.regression.linear_model import OLS
import matplotlib.pyplot as plt
from sklearn import preprocessing as pp
from linearmodels.panel import PanelOLS

In [2]:
# Set paths 
path_cleandata = os.getcwd()+'/CleanData/'

loadAppend = '_OurData_NScode'

# Data with all sorts
PortfolioData_load = path_cleandata+ 'PortfolioData' + loadAppend + '.pkl'
PortfolioData_AllSorts_BootStrap_load = path_cleandata + 'PortfolioData_AllSorts_BootStrap' + loadAppend + '.pkl'

# Centrality specific factor and portfolio data
Factors_FX_Portfolios_Cent_load = path_cleandata + 'Factors_FX_Portfolios_Cent' + loadAppend + '.pkl'
PortfolioData_PortfolioRX_AllSorts_Cent_load = path_cleandata + 'PortfolioData_PortfolioRX_AllSorts_Cent' + loadAppend + '.pkl'

# Invoice specific factor and portfolio data
Factors_FX_Portfolios_Invoice_load = path_cleandata + 'Factors_FX_Portfolios_Invoice' + loadAppend + '.pkl'
PortfolioData_PortfolioRX_AllSorts_Invoice_load = path_cleandata + 'PortfolioData_PortfolioRX_AllSorts_Invoice' + loadAppend + '.pkl'

# Function to annualize monthly mean
def mean1200(series):
    return np.mean(series,axis=0)*1200

# Function to annualize monthly std
def std12(series):
    return np.std(series,axis=0)*math.sqrt(12)*100

# Function to compute skewness
def Skew(series):
    return series.skew(axis=0)/12

# Function to compute excess kurtosis
def Kurtosis(series):
    return (series.kurtosis(axis=0)-3)/math.sqrt(12)

# Function to compute Sharpe ratio
def SharpeRatio(series):
    return np.mean(series,axis=0)/np.std(series,axis=0)*math.sqrt(12)

# Function to compute maximum drawdown
# Need to add it



In [3]:
# ----------------------------------------
# --------------| Table 1 |---------------
# ----------------------------------------

# Import the data
bycountry = pd.read_pickle(PortfolioData_load)
bycountry = bycountry[bycountry['year'].isin(list(range(1988,2016+1)))]

bycountry['logrx'] = bycountry['logrx']*1200
bycountry['forwardspread'] =bycountry['forwardspread']*1200

bycountryyear = ( bycountry.groupby(by=['iso3','year'],as_index=False)
                        .agg({'logrx':'mean', 
                        'forwardspread':'mean', 
                        'cent_exp':'mean',
                        'gdpshare':'mean', 
                        'tradetogdp':'mean'})
                )

bycountryyear.rename(columns={'cent_exp':'cent'},inplace=True)

tmp = ( bycountryyear.groupby(by='iso3',as_index=False)
                    .agg({'logrx':'mean', 
                        'forwardspread':'mean'})
       )

bycountryyear['cent'] = bycountryyear.groupby('iso3')['cent'].shift(1)
bycountryyear['gdpshare'] = bycountryyear.groupby('iso3')['gdpshare'].shift(1)
bycountryyear['tradetogdp'] = bycountryyear.groupby('iso3')['tradetogdp'].shift(1)


bycountryyear = bycountryyear.set_index(['iso3','year'])

model1 = PanelOLS.from_formula('logrx ~ pp.scale(cent) + TimeEffects', data=bycountryyear)
model2 = PanelOLS.from_formula('logrx ~ pp.scale(cent) + pp.scale(gdpshare) + TimeEffects', data=bycountryyear)
model3 = PanelOLS.from_formula('logrx ~ pp.scale(cent) + pp.scale(tradetogdp) + TimeEffects', data=bycountryyear)
model4 = PanelOLS.from_formula('forwardspread ~ pp.scale(cent) + TimeEffects', data=bycountryyear)
model5 = PanelOLS.from_formula('forwardspread ~ pp.scale(cent) + pp.scale(gdpshare) + TimeEffects', data=bycountryyear)
model6 = PanelOLS.from_formula('forwardspread ~ pp.scale(cent) + pp.scale(tradetogdp) + TimeEffects', data=bycountryyear)

print('-------------------------------------------------------------------------------')
print('------------------------ Model 1 (rx on centrality)----------------------------')
print('-------------------------------------------------------------------------------')
print(model1.fit(cov_type='clustered',cluster_entity=True))
print('------------------- Model 2 (rx on centrality + gdpshare) ---------------------')
print('-------------------------------------------------------------------------------')
print(model2.fit(cov_type='clustered',cluster_entity=True))
print('-------------------------------------------------------------------------------')
print('----------------- Model 3 (rx on centrality + tradetogdp)----------------------')
print('-------------------------------------------------------------------------------')
print(model3.fit(cov_type='clustered',cluster_entity=True))
print('-------------------------------------------------------------------------------')
print('------------------------ Model 4 (f-s on centrality) --------------------------')
print('-------------------------------------------------------------------------------')
print(model4.fit(cov_type='clustered',cluster_entity=True))
print('-------------------------------------------------------------------------------')
print('------------------ Model 5 (f-s on centrality + gdpshare) ---------------------')
print('-------------------------------------------------------------------------------')
print(model5.fit(cov_type='clustered',cluster_entity=True))
print('-------------------------------------------------------------------------------')
print('----------------- Model 6 (f-s on centrality + tradetogdp) --------------------')
print('-------------------------------------------------------------------------------')
print(model6.fit(cov_type='clustered',cluster_entity=True))


-------------------------------------------------------------------------------
------------------------ Model 1 (rx on centrality)----------------------------
-------------------------------------------------------------------------------
                          PanelOLS Estimation Summary                           
Dep. Variable:                  logrx   R-squared:                        0.0033
Estimator:                   PanelOLS   R-squared (Between):             -0.5913
No. Observations:                 695   R-squared (Within):               0.0012
Date:                Thu, May 04 2023   R-squared (Overall):             -0.0092
Time:                        11:21:26   Log-likelihood                   -2463.4
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2.1835
Entities:                          37   P-value                           0.1400
Avg Obs:                       

/opt/anaconda3/lib/python3.9/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.9/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.9/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anaconda3/lib/python3.9/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/opt/anacond

In [4]:
# ----------------------------------------
# --------------| Table 3 |---------------
# ----------------------------------------

# Import the data
portdata = pd.read_pickle(PortfolioData_AllSorts_BootStrap_load)

portdata = portdata.set_index(['sorttype','portnum'])
portdata = pd.melt(portdata,ignore_index=False)
portdata.reset_index(inplace=True)
portdata = pd.pivot(portdata,index=['sorttype','variable'],columns='portnum')
portdata.columns = portdata.columns.droplevel()
portdata.dropna()

panelAvarlist = ['prevcent','forwardspread','forwardspread_se','logrx','sdlogrx','logrx_se',
              'sharperatio','sharperatio_se','realfwdspread','realfwdspread_se',
              'conscor']

panelBvarlist = ['meanforwardfirst','fwdspread','fwdspread_se','logrx','sdlogrx','logrx_se',
              'sharperatio','sharperatio_se']

panelCvarlist = ['prevfwdspread','fwdspread','fwdspread_se','sdfwdspread','logrx','logrx_se','sdlogrx',
              'sharperatio','sharperatio_se']

panelDvarlist = ['prevSumShares','forwardspread','forwardspread_se','logrx','sdlogrx','logrx_se',
              'sharperatio','sharperatio_se','realfwdspread','realfwdspread_se']

for i in range(len(panelAvarlist)):
    panelAvarlist[i] = ('cent',panelAvarlist[i])
    
for i in range(len(panelBvarlist)):
    panelBvarlist[i] = ('unconditional',panelBvarlist[i])
    
for i in range(len(panelCvarlist)):
    panelCvarlist[i] = ('fwdspread',panelCvarlist[i])    
    
for i in range(len(panelDvarlist)):
    panelDvarlist[i] = ('invoice',panelDvarlist[i])    


panelA = portdata.loc[panelAvarlist]
panelA.loc[('cent','prevcent')]=panelA.loc[('cent','prevcent')]*100
panelB = portdata.loc[panelBvarlist]
panelC = portdata.loc[panelCvarlist]
panelD = portdata.loc[panelDvarlist]

pd.options.display.float_format = '{:,.2f}'.format

panelA.rename(columns={1:'Peripheral',4:'Central','HML':'PMC'},index={'prevcent':'prevcent*100'},inplace=True)
panelB.rename(columns={1:'Low',4:'High','HML':'UHML'},inplace=True)
panelC.rename(columns={1:'Low',4:'High','HML':'HML'},inplace=True)
panelD.rename(columns={1:'Dispersed',4:'Concentrated','HML':'DMC'},inplace=True)

print('--------------------------------------------------------------------')
print('                Panel A: Trade Network Centrality                   ')
print('--------------------------------------------------------------------')
print(panelA)
print('--------------------------------------------------------------------')
print(' ')
print('--------------------------------------------------------------------')
print('              Panel B: Unconditional Forward Spread                 ')
print('--------------------------------------------------------------------')
print(panelB)
print('--------------------------------------------------------------------')
print(' ')
print('--------------------------------------------------------------------')
print('                 Panel C: Current Forward Spread                    ')
print('--------------------------------------------------------------------')
print(panelC)
print('--------------------------------------------------------------------')
print(' ')
print('--------------------------------------------------------------------')
print('           Panel D: Currency Invoicing Concentration                ')
print('--------------------------------------------------------------------')
print(panelD)
print('--------------------------------------------------------------------')





--------------------------------------------------------------------
                Panel A: Trade Network Centrality                   
--------------------------------------------------------------------
portnum                    Peripheral    2    3  Central   PMC
sorttype variable                                             
cent     prevcent*100            0.16 0.34 0.53     0.83 -0.67
         forwardspread           2.74 1.93 1.39     0.42  2.32
         forwardspread_se        0.10 0.10 0.10     0.10  0.11
         logrx                   2.02 0.50 0.86     0.36  1.66
         sdlogrx                 6.63 7.72 6.47     6.71  4.78
         logrx_se                1.24 1.45 1.22     1.25  0.90
         sharperatio             0.30 0.06 0.13     0.05  0.35
         sharperatio_se          0.20 0.19 0.19     0.19  0.19
         realfwdspread           2.86 1.60 0.98     0.95  1.91
         realfwdspread_se        0.11 0.09 0.08     0.10  0.09
         conscor                 0.21

In [5]:
# ----------------------------------------
# --------------| Table 4 |---------------
# ----------------------------------------

# Import the data for Richmond sorts
all_factors = pd.read_pickle(Factors_FX_Portfolios_Cent_load)
all_factors.sort_values('Date',inplace=True)
filt = all_factors['UHML'].notna()
all_factors.loc[filt,'PMC_Same'] = all_factors.loc[filt]['PMC']
all_factors.loc[filt,'HML_Same'] = all_factors.loc[filt]['HML']


# all_factors is already in long form
funlist = [mean1200,std12,SharpeRatio,Skew,Kurtosis,'count']
sumstats = all_factors.agg({'PMC': funlist,
                 'HML': funlist,
                 'UHML': funlist,
                 'PMC_Same': funlist,
                 'HML_Same': funlist,
                 })

sumstats.rename(index={'mean1200':'Mean',
                          'std12':'SD',
                          'SharpeRatio':'Sharpe ratio',
                          'Skew':'Skewness',
                          'Kurtosis':'Excess kurtosis',
                          'count':'N'},inplace=True)


# Import the data for our currency invoicing sorts
all_factors = pd.read_pickle(Factors_FX_Portfolios_Invoice_load)
all_factors.sort_values('Date',inplace=True)
filt = all_factors['UHML'].notna()
all_factors.loc[filt,'DMC_Same'] = all_factors.loc[filt]['DMC']
all_factors.loc[filt,'HML_Same'] = all_factors.loc[filt]['HML']


# all_factors is already in long form
funlist = [mean1200,std12,SharpeRatio,Skew,Kurtosis,'count']
sumstats_invoice = all_factors.agg({'DMC': funlist,
                 'HML': funlist,
                 'UHML': funlist,
                 'DMC_Same': funlist,
                 'HML_Same': funlist,
                 })

sumstats_invoice.rename(index={'mean1200':'Mean',
                          'std12':'SD',
                          'SharpeRatio':'Sharpe ratio',
                          'Skew':'Skewness',
                          'Kurtosis':'Excess kurtosis',
                          'count':'N'},inplace=True)


In [6]:
sumstats

,PMC,HML,UHML,PMC_Same,HML_Same
Mean,1.66,5.41,2.31,2.15,4.61
SD,4.78,7.14,6.55,4.47,7.07
Sharpe ratio,0.35,0.76,0.35,0.48,0.65
Skewness,-0.03,-0.05,-0.07,-0.07,-0.05
Excess kurtosis,-0.10,-0.66,0.14,-0.09,-0.56
N,336.00,347.00,180.00,180.00,180.00


In [7]:
# ----------------------------------------
# --------------| Table 5 |---------------
# ----------------------------------------

# Import the data for Richmond sorts
all_factors = pd.read_pickle(Factors_FX_Portfolios_Cent_load)
all_factors.sort_values('Date',inplace=True)
all_factors['PMC'] = all_factors['PMC']*1200
all_factors['HML'] = all_factors['HML']*1200
all_factors['UHML'] = all_factors['UHML']*1200
filt = all_factors['UHML'].notna()
all_factors.loc[filt,'HML2'] = all_factors.loc[filt]['HML']

all_factors = all_factors.set_index('Date')

modelhml = OLS.from_formula('HML ~ PMC',data=all_factors)
modelunconditional = OLS.from_formula('UHML ~ PMC',data=all_factors)
#modelhml2 = OLS.from_formula('HML2 ~ PMC',data=all_factors)

print('------------------------------------------------------------------------------')
print('                           Model HML on PMC                                   ')
print('------------------------------------------------------------------------------')
print(modelhml.fit(cov_type='HAC',cov_kwds={'maxlags':12},use_t=True).summary())
print(' ')
print('------------------------------------------------------------------------------')
print('                           Model UHML on PMC                                  ')
print('------------------------------------------------------------------------------')
print(modelunconditional.fit(cov_type='HAC',cov_kwds={'maxlags':12},use_t=True).summary())

# Import the data for our currency invoicing sorts
all_factors = pd.read_pickle(Factors_FX_Portfolios_Invoice_load)
all_factors.sort_values('Date',inplace=True)
all_factors['DMC'] = all_factors['DMC']*1200
all_factors['HML'] = all_factors['HML']*1200
all_factors['UHML'] = all_factors['UHML']*1200
filt = all_factors['UHML'].notna()
all_factors.loc[filt,'HML2'] = all_factors.loc[filt]['HML']

all_factors = all_factors.set_index('Date')

modelhml = OLS.from_formula('HML ~ DMC',data=all_factors)
modelunconditional = OLS.from_formula('UHML ~ DMC',data=all_factors)
#modelhml2 = OLS.from_formula('HML2 ~ DMC',data=all_factors)

print('------------------------------------------------------------------------------')
print('                            Model HML on DMC                                  ')
print('------------------------------------------------------------------------------')
print(modelhml.fit(cov_type='HAC',cov_kwds={'maxlags':12},use_t=True).summary())
print(' ')
print('------------------------------------------------------------------------------')
print('                            Model UHML on DMC                                 ')
print('------------------------------------------------------------------------------')
print(modelunconditional.fit(cov_type='HAC',cov_kwds={'maxlags':12},use_t=True).summary())




------------------------------------------------------------------------------
                           Model HML on PMC                                   
------------------------------------------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                    HML   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     1.962
Date:                Thu, 04 May 2023   Prob (F-statistic):              0.162
Time:                        11:21:28   Log-Likelihood:                -1546.7
No. Observations:                 336   AIC:                             3097.
Df Residuals:                     334   BIC:                             3105.
Df Model:                           1                                         
Covariance Type:                  HAC               

In [19]:
# Added by Nick: I like horse races!

all_factors_l = pd.read_pickle(Factors_FX_Portfolios_Cent_load)
all_factors_r = pd.read_pickle(Factors_FX_Portfolios_Invoice_load)
all_factors = pd.merge(all_factors_r,all_factors_l[['Date','PMC']],on='Date',how='left')

all_factors.sort_values('Date',inplace=True)
all_factors['DMC'] = all_factors['DMC']*1200
all_factors['PMC'] = all_factors['PMC']*1200
all_factors['HML'] = all_factors['HML']*1200
all_factors['UHML'] = all_factors['UHML']*1200
filt = all_factors['UHML'].notna()
all_factors.loc[filt,'HML2'] = all_factors.loc[filt]['HML']

modelhml = OLS.from_formula('HML ~ DMC+PMC',data=all_factors)
modelunconditional = OLS.from_formula('UHML ~ DMC+PMC',data=all_factors)


print('------------------------------------------------------------------------------')
print('                            Model HML on DMC                                  ')
print('------------------------------------------------------------------------------')
print(modelhml.fit(cov_type='HAC',cov_kwds={'maxlags':12},use_t=True).summary())
print(' ')
print('------------------------------------------------------------------------------')
print('                            Model UHML on DMC                                 ')
print('------------------------------------------------------------------------------')
print(modelunconditional.fit(cov_type='HAC',cov_kwds={'maxlags':12},use_t=True).summary())

------------------------------------------------------------------------------
                            Model HML on DMC                                  
------------------------------------------------------------------------------
                            OLS Regression Results                            
Dep. Variable:                    HML   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     51.74
Date:                Thu, 04 May 2023   Prob (F-statistic):           2.79e-19
Time:                        11:24:58   Log-Likelihood:                -1015.0
No. Observations:                 233   AIC:                             2036.
Df Residuals:                     230   BIC:                             2046.
Df Model:                           2                                         
Covariance Type:                  HAC               

In [8]:
# ----------------------------------------
# --------------| Table 6 |---------------
# ----------------------------------------

# Import the data for Richmond sorts
portdata = pd.read_pickle(PortfolioData_PortfolioRX_AllSorts_Cent_load)
facdata = pd.read_pickle(Factors_FX_Portfolios_Cent_load)

portdata.set_index('Date',inplace=True)
facdata.set_index('Date',inplace=True)

alldata = pd.merge(left=portdata,right=facdata,on='Date',how='left')
alldata['logrx'] = alldata['logrx']*1200
alldata['PMC'] = alldata['PMC']*1200
filt = alldata['portnum'] != 'HML'
alldata = alldata.loc[filt]
alldata = alldata[['type','portnum','logrx','PMC']]
alldata.sort_values('Date',inplace=True)

# Start by running separate regressions for each portfolio (there might be a cleaner way to do this as in R)
facs = ['fs','unconditional','cent']
variables = ['alpha_i','alpha_i_se','beta_i','beta_i_se','AdjR2','NumObs']
portfolios = [1,2,3,4]
table_PMC = np.zeros((18, 4))
iterables = [facs, variables]
index = pd.MultiIndex.from_product(iterables, names=["Sorting", "Variable"])
table_PMC = pd.DataFrame(table_PMC,index=index,columns=[1,2,3,4])
print('------------------- Regression on PMC factor ---------------------------')
i=0
for fac in facs:
    filt = alldata['type'] == fac
    data = alldata.loc[filt]
    data = data[['portnum','logrx','PMC']]
    for pf in portfolios:
        filt_pf = data['portnum'] == pf
        data_pf = data.loc[filt_pf]
        model = OLS.from_formula('logrx ~ PMC',data=data_pf).fit(cov_type='HAC',cov_kwds={'maxlags':12},use_t=True)
        print('Sorting on '+fac+ f' and portfolio # {pf}')
        print(model.summary())
        pars = model.params
        adjR2 = model.rsquared_adj
        stde = model.bse
        Nobs = model.nobs
        table_PMC.loc[(fac,'alpha_i'),pf] = pars[0]
        table_PMC.loc[(fac,'alpha_i_se'),pf] = stde[0]
        table_PMC.loc[(fac,'beta_i'),pf] = pars[1]
        table_PMC.loc[(fac,'beta_i_se'),pf] = stde[1]
        table_PMC.loc[(fac,'AdjR2'),pf] = adjR2
        table_PMC.loc[(fac,'NumObs'),pf] = Nobs
        
    i+=6
        

# Import the data for our invoicing sorts
portdata = pd.read_pickle(PortfolioData_PortfolioRX_AllSorts_Invoice_load)
facdata = pd.read_pickle(Factors_FX_Portfolios_Invoice_load)

portdata.set_index('Date',inplace=True)
facdata.set_index('Date',inplace=True)

alldata = pd.merge(left=portdata,right=facdata,on='Date',how='left')
alldata['logrx'] = alldata['logrx']*1200
alldata['DMC'] = alldata['DMC']*1200
filt = alldata['portnum'] != 'HML'
alldata = alldata.loc[filt]
alldata = alldata[['type','portnum','logrx','DMC']]
alldata.sort_values('Date',inplace=True)

# Start by running separate regressions for each portfolio (there might be a cleaner way to do this as in R)
facs = ['fs','unconditional','invoice']
variables = ['alpha_i','alpha_i_se','beta_i','beta_i_se','AdjR2','NumObs']
portfolios = [1,2,3,4]
table_DMC = np.zeros((18, 4))
iterables = [facs, variables]
index = pd.MultiIndex.from_product(iterables, names=["Sorting", "Variable"])
table_DMC = pd.DataFrame(table_PMC,index=index,columns=[1,2,3,4])
print('------------------- Regression on DMC factor ---------------------------')
for fac in facs:
    filt = alldata['type'] == fac
    data = alldata.loc[filt]
    data = data[['portnum','logrx','DMC']]
    for pf in portfolios:
        filt_pf = data['portnum'] == pf
        data_pf = data.loc[filt_pf]
        model = OLS.from_formula('logrx ~ DMC',data=data_pf).fit(cov_type='HAC',cov_kwds={'maxlags':12},use_t=True)
        print('Sorting on '+fac+ f' and portfolio # {pf}')
        print(model.summary())
        pars = model.params
        adjR2 = model.rsquared_adj
        stde = model.bse
        Nobs = model.nobs
        table_DMC.loc[(fac,'alpha_i'),pf] = pars[0]
        table_DMC.loc[(fac,'alpha_i_se'),pf] = stde[0]
        table_DMC.loc[(fac,'beta_i'),pf] = pars[1]
        table_DMC.loc[(fac,'beta_i_se'),pf] = stde[1]
        table_DMC.loc[(fac,'AdjR2'),pf] = adjR2
        table_DMC.loc[(fac,'NumObs'),pf] = Nobs
        
    i+=6     


------------------- Regression on PMC factor ---------------------------
Sorting on fs and portfolio # 1
                            OLS Regression Results                            
Dep. Variable:                  logrx   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                 0.0003533
Date:                Thu, 04 May 2023   Prob (F-statistic):              0.985
Time:                        11:21:33   Log-Likelihood:                -1502.8
No. Observations:                 336   AIC:                             3010.
Df Residuals:                     334   BIC:                             3017.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [9]:
table_DMC

1      2      3      4
Sorting       Variable                              
fs            alpha_i     -1.96  -1.02   2.00   3.43
              alpha_i_se   1.48   1.47   1.89   2.23
              beta_i      -0.05   0.11   0.16   0.27
              beta_i_se    0.09   0.07   0.09   0.14
              AdjR2       -0.00   0.02   0.03   0.05
              NumObs     233.00 233.00 233.00 233.00
unconditional alpha_i      0.42   1.02   1.55   2.16
              alpha_i_se   1.46   0.83   2.90   2.53
              beta_i       0.05   0.08   0.46   0.56
              beta_i_se    0.10   0.03   0.17   0.17
              AdjR2       -0.00   0.03   0.07   0.11
              NumObs     180.00 180.00 180.00 180.00
invoice       alpha_i      1.50   2.53   1.16   1.50
              alpha_i_se   2.19   2.68   2.41   2.19
              beta_i       0.55   0.33   0.19  -0.45
              beta_i_se    0.18   0.14   0.12   0.18
              AdjR2        0.19   0.05   0.02   0.13
              NumObs     233.00 233.00 233.00 233.00

In [20]:
table_PMC

1      2      3      4
Sorting       Variable                              
fs            alpha_i     -1.79  -0.41   3.21   2.73
              alpha_i_se   1.26   1.33   1.77   1.86
              beta_i       0.00   0.03  -0.13   0.36
              beta_i_se    0.16   0.18   0.30   0.37
              AdjR2       -0.00  -0.00   0.00   0.03
              NumObs     336.00 336.00 336.00 336.00
unconditional alpha_i     -0.20   0.67  -0.33  -0.24
              alpha_i_se   1.37   0.80   2.62   1.92
              beta_i       0.32   0.20   1.11   1.41
              beta_i_se    0.13   0.06   0.26   0.18
              AdjR2        0.06   0.11   0.25   0.41
              NumObs     180.00 180.00 180.00 180.00
cent          alpha_i      1.23   0.26   0.77   1.23
              alpha_i_se   1.39   1.66   1.38   1.39
              beta_i       0.48   0.14   0.06  -0.52
              beta_i_se    0.26   0.26   0.20   0.26
              AdjR2        0.12   0.01  -0.00   0.14
              NumObs     336.00 336.00 336.00 336.00